In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)

TensorFlow: 2.19.0
NumPy: 2.0.2
Pandas: 2.2.2


In [ ]:
!pip install -q datasets sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 3.7 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sacrebleu import corpus_bleu

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [ ]:
ds = load_dataset("ai4bharat/samanantar", "hi")

# Take subset for feasibility
df = ds["train"].select(range(50000)).to_pandas()
df = df[["src", "tgt"]]

print("Initial size:", len(df))
df.head()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

hi/train-00000-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

hi/train-00001-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

hi/train-00002-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

...

hi/train-00006-of-00008.parquet:   0%|          | 0.00/239M [00:00<?, ?B/s]

hi/train-00007-of-00008.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10125706 [00:00<?, ? examples/s]

Initial size: 50000


,src,tgt
0,"However, Paes, who was partnering Australia's ...",आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...
1,"Whosoever desires the reward of the world, wit...",और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...
2,The value of insects in the biosphere is enorm...,"जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ..."
3,Mithali To Anchor Indian Team Against Australi...,आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
4,After the assent of the Honble President on 8t...,"8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व..."


In [ ]:
def clean_text(df):
    df = df.copy()
    df["src"] = df["src"].astype(str).str.strip()
    df["tgt"] = df["tgt"].astype(str).str.strip()

    df = df[df["src"].str.split().str.len().between(2, 80)]
    df = df[df["tgt"].str.split().str.len().between(2, 80)]
    return df

df = clean_text(df)
print("Cleaned size:", len(df))

Cleaned size: 49600


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df["tgt"] = "<start> " + train_df["tgt"] + " <end>"
test_df["tgt"]  = "<start> " + test_df["tgt"]  + " <end>"

In [ ]:
MAX_VOCAB = 30000
MAX_LEN = 80

src_tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<unk>")
tgt_tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<unk>")

src_tokenizer.fit_on_texts(train_df["src"])
tgt_tokenizer.fit_on_texts(train_df["tgt"])

X_src = pad_sequences(
    src_tokenizer.texts_to_sequences(train_df["src"]),
    maxlen=MAX_LEN, padding="post", truncating="post"
)

Y_seq = tgt_tokenizer.texts_to_sequences(train_df["tgt"])

Y_in = pad_sequences(
    [y[:-1] for y in Y_seq],
    maxlen=MAX_LEN, padding="post"
)

Y_out = pad_sequences(
    [y[1:] for y in Y_seq],
    maxlen=MAX_LEN, padding="post"
)

Y_out = np.expand_dims(Y_out, -1)

print(X_src.shape, Y_in.shape, Y_out.shape)

(39680, 80) (39680, 80) (39680, 80, 1)


In [ ]:
# cell 8 build seq2seq LSTM
encoder_inputs = Input(shape=(MAX_LEN,))
enc_emb = Embedding(MAX_VOCAB, 128, mask_zero=True)(encoder_inputs)
_, h, c = LSTM(128, return_state=True)(enc_emb)

decoder_inputs = Input(shape=(MAX_LEN,))
dec_emb = Embedding(MAX_VOCAB, 128, mask_zero=True)(decoder_inputs)
dec_lstm = LSTM(128, return_sequences=True, return_state=True)
dec_outputs, _, _ = dec_lstm(dec_emb, initial_state=[h, c])

dec_dense = Dense(MAX_VOCAB, activation="softmax")
dec_outputs = dec_dense(dec_outputs)

lstm_model = Model([encoder_inputs, decoder_inputs], dec_outputs)
lstm_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy"
)

lstm_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 80, 128)   │  3,840,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 80)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 80, 128)   │  3,840,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │    131,584 │ embedding[0][0],  │
│                     │ (None, 128),      │            │ not_equal[0][0]   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 80, 128), │    131,584 │ embedding_1[0][0… │
│                     │ (None, 128),      │            │ lstm[0][1],       │
│                     │ (None, 128)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 80, 30000) │  3,870,000 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,813,168 (45.06 MB)

 Trainable params: 11,813,168 (45.06 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
  # training
history = lstm_model.fit(
    [X_src, Y_in],
    Y_out,
    batch_size=32,
    epochs=3,
    validation_split=0.1
)

Epoch 1/3
 150/1116 ━━━━━━━━━━━━━━━━━━━━ 52:52 3s/step - loss: 8.7106

KeyboardInterrupt: 

In [ ]:
# plotting training loss
plt.figure(figsize=(8,5))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("LSTM Training Loss")
plt.legend()
plt.show()

NameError: name 'history' is not defined

<Figure size 800x500 with 0 Axes>

In [ ]:
encoder_model = Model(encoder_inputs, [h, c])

dec_state_h = Input(shape=(128,))
dec_state_c = Input(shape=(128,))
dec_states = [dec_state_h, dec_state_c]

dec_outputs2, h2, c2 = dec_lstm(
    dec_emb, initial_state=dec_states
)
dec_outputs2 = dec_dense(dec_outputs2)

decoder_model = Model(
    [decoder_inputs] + dec_states,
    [dec_outputs2, h2, c2]
)

In [ ]:
def decode_sequence(input_seq):
    states = encoder_model.predict(input_seq, verbose=0)
    target = np.zeros((1, 1))
    decoded = []

    for _ in range(MAX_LEN):
        preds, h, c = decoder_model.predict([target] + states, verbose=0)
        idx = np.argmax(preds[0, -1, :])
        word = tgt_tokenizer.index_word.get(idx, "")
        if word == "<end>" or idx == 0:
            break
        decoded.append(word)
        target[0, 0] = idx
        states = [h, c]

    return " ".join(decoded)

In [ ]:
sample = test_df.sample(200, random_state=42)

preds, refs = [], []

for _, row in sample.iterrows():
    seq = pad_sequences(
        src_tokenizer.texts_to_sequences([row["src"]]),
        maxlen=MAX_LEN, padding="post"
    )
    preds.append(decode_sequence(seq))
    refs.append([row["tgt"]])

bleu = corpus_bleu(preds, refs).score
print("Baseline LSTM BLEU Score:", bleu)

NameError: name 'test_df' is not defined

In [ ]:
import sacrebleu

def compute_all_metrics(preds, refs):
    bleu = sacrebleu.corpus_bleu(preds, refs).score
    meteor = sacrebleu.corpus_meteor(preds, refs).score
    chrf = sacrebleu.corpus_chrf(preds, refs).score

    return {
        "BLEU": round(bleu, 2),
        "METEOR": round(meteor, 2),
        "chrF": round(chrf, 2)
    }

metrics_lstm = compute_all_metrics(preds, refs)
print("LSTM Metrics:", metrics_lstm)

ModuleNotFoundError: No module named 'sacrebleu'

In [ ]:
results_df = pd.DataFrame({
    "Model": ["LSTM Seq2Seq (Baseline)"],
    "Dataset": ["Samanantar"],
    "Language Pair": ["EN-HI"],
    "BLEU Score": [bleu]
})

results_df

,Model,Dataset,Language Pair,BLEU Score
0,LSTM Seq2Seq (Baseline),Samanantar,EN-HI,0.336385
